<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/lease_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
def lease_findifNull(lo_left,lo_right,la_left,la_right,research,zip):
  result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {})'.format(lo_left, lo_right, la_left, la_right, zip))
  lis = []
  count = 0
  di ={}
  df = pd.DataFrame()
  for item in result:
    if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1
            di = item
  for key in di:
    lis.append(key)
  if(count >= 5): #not posible larger than 5 at the first time
    df = df.T
    df.columns = lis
    return df
  else:
    lo_left -= 0.008
    lo_right += 0.008
    la_left -= 0.008
    la_right += 0.008
    return lease_findifNull(lo_left, lo_right, la_left, la_right, research, zip)


In [15]:
def lease_findHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,house_leftSqt,house_rightSqt,research,zip,a):
  result = research.search(resource = 'Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {}),(LivingArea = {}+),(LivingArea{}-)'.format(house_leftLo, house_rightLo, house_leftLa, house_rightLa, zip,house_leftSqt,house_rightSqt))
  basic = house_leftSqt
  count = 0
  di={}
  lis =[]
  df = pd.DataFrame()
  for item in result:
    if(item['PropertyType'] == 'Residential Lease' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1
            di = item # 没这行
    for key in di: #没这行
      lis.append(key) #没这行
    if(count >= 5):  
      df = df.T
      df.columns = lis # =column
    return df
  else:
    house_leftLo -= 0.008
    house_rightLo += 0.008
    house_leftLa -= 0.008
    house_rightLa += 0.008

    #living area
    if(a == 0):
      if(basic <= 1000):
        house_leftSqt -=100
        house_rightSqt +=100
      elif(basic <= 2200):
        house_leftSqt -=200
        house_rightSqt +=200
      else:
        house_leftSqt -=300
        house_rightSqt +=300
      a += 1

  return lease_findHouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,house_leftSqt,house_rightSqt,research,zip,a)

In [25]:
def insertId(pulled_value, mls, rets_client):
  column = []
  for key in pulled_value:
    column.append(key)
  print("\nThis value exists in Website")
  zip = pulled_value.get('PostalCode')
  house_leftLa = float(pulled_value.get('Latitude'))
  house_rightLa = float(pulled_value.get('Latitude')) 
  house_leftLo = float(pulled_value.get('Longitude'))
  house_rightLo = float(pulled_value.get('Longitude'))
  house_leftSqt = float(pulled_value.get('LivingArea'))
  house_rightSqt = float(pulled_value.get('LivingArea'))
  house_bed = pulled_value.get("BedroomsTotal")
  house_bath = pulled_value.get("BathroomsTotalInteger")

  result1 = lease_findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip)        
  result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
  'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
  'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
  'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
  'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
  'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
  'USProperty_MUI', 'YearBuilt']]
  price=[]
  for item in result1['ClosePrice']:
    item = float(item)
    price.append(item)
    result1['ClosePrice'] = price
    mean1 = result1['ClosePrice'].mean()
    top1 = list(result1['ClosePrice'])
    top1.sort(reverse = True)
    print('The mean for {} is {}'.format(mls, mean1))
    print('The top 3 for {} are {}'.format(mls, top1))
    return result1.to_csv('{}.csv'.format(mls))

In [29]:
def findAdd(address, city, state, zip, rets_client):
  result = cg.address(address, city = city, state= state,zipcode = zip)
  lo_left = result[0]['coordinates']['x']
  lo_right = result[0]['coordinates']['x']
  la_left = result[0]['coordinates']['y']
  la_right = result[0]['coordinates']['y']
  result = lease_findifNull(lo_left, lo_right, la_left, la_right, rets_client, zip)
  result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
  'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
  'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
  'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
  'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
  'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
  'USProperty_MUI', 'YearBuilt']]

  price = []
  for item in result['ClosePrice']:
    item = float(item)
    price.append(item)
    result['ClosePrice'] = price
    mean = result['ClosePrice'].mean()
    top = list(result['ClosePrice'])
    top.sort(reverse = True)
    top_mean = sum(top)/ len(top) # new added
    print('The Avg for {}, {}, {} is {}'.format(address, state, zip, mean))
    print('The top 3 for {}, {}, {} are {}'.format(address, state, zip, top))
    print('The top 3 Avg for {}, {}, {} is {}').format(address, state, zip, top_mean) #new added
    return result.to_csv('{},{},{}.csv'.format(address, state, zip))


    

In [ ]:
def get(i,finalN,number,code,rets_client):
  pulled_value = {}
  result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number[i],finalN[i],code[i]))

In [22]:
def main():
  login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
  username = '0671181_NID'
  password = 'Rt$tg6jx'
  rets_client = Session(login_url, username, password)
  rets_client.login()

  print("enter mls:")
  mls = input()
  for i in mls:
    result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(ListingId = {})'.format(mls))
    return

In [23]:
if __name__ == "__main__":
    print(main())

enter mls:
14730621
None
